# Step by Step and Error Aware Yöntem DB

In [ ]:
import json
import torch
import gc
from pathlib import Path
from method_db_helper import MethodDatabaseBuilder

PROMPTS = {
    "step_by_step": """Analyze the following mathematical problem and its solution.

Problem: {question}

Solution: {solution}

Task: Extract a general step-by-step method that can be applied to similar problems. Focus on the procedure, not the specific numbers.

Provide your answer as a JSON object with this exact format:
{{"method": "General step-by-step approach description (3-5 sentences)"}}""",

    "error_aware": """Analyze the following mathematical problem and its solution.

Problem: {question}

Solution: {solution}

Task: Describe the general solution method and highlight common mistakes to avoid in similar problems.

Provide your answer as a JSON object with this exact format:
{{"method": "Correct approach and common errors to avoid (4-6 sentences)"}}"""
}

# =============================================================================
# CONFIGURATION
# =============================================================================
MODEL_ID = "ytu-ce-cosmos/Turkish-Gemma-9b-T1"
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

INPUT_FILE = Path("/content/selected_200_questions.json")
OUTPUT_BASE_DIR = Path("method_databases")
OUTPUT_BASE_DIR.mkdir(parents=True, exist_ok=True)

MAX_NEW_TOKENS = 4096
DTYPE = torch.bfloat16



# =============================================================================
# MAIN EXECUTION
# =============================================================================
def main():
    print("\n" + "="*80)
    print("METHOD DATABASE BUILDER")
    print("="*80)

    # Load questions
    print(f"\n📥 Loading questions from: {INPUT_FILE}")
    with open(INPUT_FILE, "r", encoding="utf-8") as f:
        questions_data = json.load(f)

    print(f"✅ Loaded {len(questions_data)} questions")

    # Initialize builder
    builder = MethodDatabaseBuilder(MODEL_ID, EMBEDDING_MODEL)

    # Build database for each prompt
    for prompt_name, prompt_template in PROMPTS.items():
        try:
            # Build
            index, metadata = builder.build_database(
                prompt_name,
                prompt_template,
                questions_data
            )

            # Save
            builder.save_database(
                prompt_name,
                index,
                metadata,
                OUTPUT_BASE_DIR
            )

            # Cleanup
            torch.cuda.empty_cache()
            gc.collect()

        except Exception as e:
            print(f"\n❌ Error building database for {prompt_name}: {e}")
            continue

    print("\n" + "="*80)
    print("✅ ALL DATABASES BUILT SUCCESSFULLY")
    print(f"📂 Output directory: {OUTPUT_BASE_DIR}")
    print("="*80)

    # Final cleanup
    del builder.model
    torch.cuda.empty_cache()
    gc.collect()


if __name__ == "__main__":
    main()


METHOD DATABASE BUILDER

📥 Loading questions from: /content/selected_200_questions.json
✅ Loaded 200 questions
🔧 Initializing Method Database Builder...

📥 Loading reasoning model: ytu-ce-cosmos/Turkish-Gemma-9b-T1


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

📥 Loading embedding model: sentence-transformers/all-MiniLM-L6-v2
✅ Embedding dimension: 384

Building database for prompt: step_by_step



Generating methods (step_by_step):   0%|          | 1/200 [02:17<7:36:41, 137.69s/it]


Example 1:
Solution: Define a variable ** Let x be the number of cans Tim has at first.
After Jeff comes by, how many cans of soda does Tim have left? ** After Jeff comes by, Tim has x-6 cans left.
How many more cans does Tim buy? ** Tim buys another (x-6)/2 cans.
Write an equation ** x-6+(x-6)/2=24
Simplify ** 2*x-12+x-6=48
Simplify ** 3*x-18=48
Simplify ** 3*x=66
Simplify ** x=<<22=22>>22
#### 22
Method: General step-by-step approach description (3-5 sentences)



...


Example 10:
Solution: How many units of each fruit did I buy? ** If 36 fruits were evenly split between 3 types of fruits, then I bought 36/3 = <<36/3=12>>12 units of each fruit
How much do 12 oranges cost? ** If 1 orange costs $0.50 then 12 oranges will cost $0.50 * 12 = $<<0.5*12=6>>6
How much did I spend on the other 2 fruit types? ** If my total bill was $66 and I spent $6 on oranges then I spent $66 - $6 = $<<66-6=60>>60 on the other 2 fruit types.
How much does 1 watermelon cost? ** Assuming the price of watermelon is W, and knowing that you can buy 4 apples for the same price and that the price of one apple is A, then 1W=4A
How much did we spend? ** If we know we bought 12 watermelons and 12 apples for $60, then we know that $60 = 12W + 12A
How much did we spend? ** Knowing that 1W=4A, then we can convert the above to $60 = 12(4A) + 12A
How much did we spend? ** $60 = 48A + 12A
How much did we spend? ** $60 = <<60=60>>60A
How much does one apple cost? ** Then we know the price 

Generating methods (error_aware): 100%|██████████| 200/200 [3:31:36<00:00, 63.48s/it]


🔢 Creating embeddings for 200 methods...


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

🔍 Building FAISS index...
✅ Database built: 200 vectors indexed
💾 Database saved to: method_databases/error_aware
   - FAISS index: method_databases/error_aware/faiss_index.bin
   - Metadata: method_databases/error_aware/metadata.json
   - Summary: method_databases/error_aware/summary.json

✅ ALL DATABASES BUILT SUCCESSFULLY
📂 Output directory: method_databases


# Pattern and Conceptual Yöntem DB

In [6]:
PROMPTS = {
    "pattern": """Analyze the following mathematical problem and its solution.

Problem: {question}

Solution: {solution}

Task: Identify the problem-solving pattern or structure used. Describe how to recognize similar problems.

Provide your answer as a JSON object with this exact format:
{{"method": "Description of the solution pattern (3-5 sentences)"}}""",

    "conceptual": """Analyze the following mathematical problem and its solution.

Problem: {question}

Solution: {solution}

Task: Explain the underlying mathematical principle or concept that makes this solution work. Focus on WHY these steps work, not just what they are.

Provide your answer as a JSON object with this exact format:
{{"method": "Explanation of the core mathematical principle (3-5 sentences)"}}"""
}

# =============================================================================
# MAIN EXECUTION
# =============================================================================
def main():
    print("\n" + "="*80)
    print("METHOD DATABASE BUILDER")
    print("="*80)

    # Load questions
    print(f"\n📥 Loading questions from: {INPUT_FILE}")
    with open(INPUT_FILE, "r", encoding="utf-8") as f:
        questions_data = json.load(f)

    print(f"✅ Loaded {len(questions_data)} questions")

    # Initialize builder
    builder = MethodDatabaseBuilder(MODEL_ID, EMBEDDING_MODEL)

    # Build database for each prompt
    for prompt_name, prompt_template in PROMPTS.items():
        try:
            # Build
            index, metadata = builder.build_database(
                prompt_name,
                prompt_template,
                questions_data
            )

            # Save
            builder.save_database(
                prompt_name,
                index,
                metadata,
                OUTPUT_BASE_DIR
            )

            # Cleanup
            torch.cuda.empty_cache()
            gc.collect()

        except Exception as e:
            print(f"\n❌ Error building database for {prompt_name}: {e}")
            continue

    print("\n" + "="*80)
    print("✅ ALL DATABASES BUILT SUCCESSFULLY")
    print(f"📂 Output directory: {OUTPUT_BASE_DIR}")
    print("="*80)

    # Final cleanup
    del builder.model
    torch.cuda.empty_cache()
    gc.collect()


if __name__ == "__main__":
    main()


METHOD DATABASE BUILDER

📥 Loading questions from: /content/selected_200_questions.json
✅ Loaded 200 questions
🔧 Initializing Method Database Builder...

📥 Loading reasoning model: ytu-ce-cosmos/Turkish-Gemma-9b-T1


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/853 [00:00<?, ?B/s]

...


Example 10:
Solution: How many units of each fruit did I buy? ** If 36 fruits were evenly split between 3 types of fruits, then I bought 36/3 = <<36/3=12>>12 units of each fruit
How much do 12 oranges cost? ** If 1 orange costs $0.50 then 12 oranges will cost $0.50 * 12 = $<<0.5*12=6>>6
How much did I spend on the other 2 fruit types? ** If my total bill was $66 and I spent $6 on oranges then I spent $66 - $6 = $<<66-6=60>>60 on the other 2 fruit types.
How much does 1 watermelon cost? ** Assuming the price of watermelon is W, and knowing that you can buy 4 apples for the same price and that the price of one apple is A, then 1W=4A
How much did we spend? ** If we know we bought 12 watermelons and 12 apples for $60, then we know that $60 = 12W + 12A
How much did we spend? ** Knowing that 1W=4A, then we can convert the above to $60 = 12(4A) + 12A
How much did we spend? ** $60 = 48A + 12A
How much did we spend? ** $60 = <<60=60>>60A
How much does one apple cost? ** Then we know the price 

Generating methods (conceptual): 100%|██████████| 200/200 [2:45:00<00:00, 49.50s/it]


🔢 Creating embeddings for 200 methods...


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

🔍 Building FAISS index...
✅ Database built: 200 vectors indexed
💾 Database saved to: method_databases/conceptual
   - FAISS index: method_databases/conceptual/faiss_index.bin
   - Metadata: method_databases/conceptual/metadata.json
   - Summary: method_databases/conceptual/summary.json

✅ ALL DATABASES BUILT SUCCESSFULLY
📂 Output directory: method_databases
